In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
import warnings
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from sklearn.preprocessing import MinMaxScaler


warnings.filterwarnings(action='ignore')

In [ ]:
tas = pd.read_csv('../input/chile-temperature-daily-stations-2018/cr2_tasDaily_2018.csv')
tas.head(15)

In [ ]:
for n in range(0,14):
        print(tas['codigo_estacion'][n],':',tas['330021'][n])

In [ ]:
def data_processing(df,station):
    df = df[14:] #drop
    df=df.replace(float(-9999),np.nan)
    df=df.replace('-9999',np.nan)
    print(df['01202012'].isna().sum(),' NaN values of ',df.shape[0])
    df = df[['codigo_estacion',station]]
    df['codigo_estacion'] =  pd.to_datetime(df['codigo_estacion'], format='%Y-%m-%d')
    df=df.dropna().reset_index().drop(columns=['index']).rename(columns={'codigo_estacion':'datetime'}).set_index('datetime')
    
    return df

In [ ]:
df=data_processing(tas,'330021')

In [ ]:
plt.style.use('fivethirtyeight')
# Color pallete for plotting
color_pal = ["#F8766D", "#D39200", "#93AA00",
             "#00BA38", "#00C19F", "#00B9E3",
             "#619CFF", "#DB72FB"]
df.plot(style='.', figsize=(15,15), color=color_pal[0], title='Temperature')
plt.show()

In [ ]:
split_date = datetime.date(2002, 1, 1).isoformat()
X_train = df.loc[df.index <= split_date].copy()
X_test = df.loc[df.index > split_date].copy()

In [ ]:
X_test \
.rename(columns={'330021': 'TEST SET'}) \
.join(X_train.rename(columns={'330021': 'TRAINING SET'}),how='outer') \
.plot(figsize=(15,5), title='Temperature Chile station', style='.')
plt.show()

In [ ]:
X_train = X_train.reset_index().rename(columns={'datetime':'ds','330021':'y' })
X_test = X_test.reset_index().rename(columns={'datetime':'ds','330021':'y' })

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
model = Prophet()
model.fit(X_train)

tas_predict = model.predict(X_test)
tas_predict.head()


In [ ]:
# Plot the forecast
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = model.plot(tas_predict,
                 ax=ax)
plt.show()

In [ ]:
# Plot the components of the model
fig = model.plot_components(tas_predict)

# Compare Forecast results with real values of temperature.

In [ ]:
# Plot the forecast with the actuals
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(X_test['ds'], X_test['y'], color='r')
fig = model.plot(tas_predict, ax=ax)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print('RMSE '+ str(round(mean_squared_error(y_true=X_test['y'],y_pred=tas_predict['yhat']),3)))
print('MAE '+ str(round(mean_absolute_error(y_true=X_test['y'], y_pred=tas_predict['yhat']),3)))
print('MAPE ' +str(round(mean_absolute_percentage_error(y_true=X_test['y'], y_pred=tas_predict['yhat']),3))+'%')

# Neuronal Network

In [ ]:
PASOS=7

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
 
# load dataset
values = df.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
values=values.reshape(-1, 1) # esto lo hacemos porque tenemos 1 sola dimension
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, PASOS, 1)
reframed.head()

In [ ]:
reframed.shape

In [ ]:
# split into train and test sets
values = reframed.values
n_train_days = 12612
train = values[:n_train_days, :]
test = values[n_train_days:, :]
# split into input and outputs
x_train, y_train = train[:, :-1], train[:, -1]
x_val, y_val = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_val = x_val.reshape((x_val.shape[0], 1, x_val.shape[1]))
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)


In [ ]:
def crear_modeloFF():
    model = Sequential() 
    model.add(Dense(PASOS, input_shape=(1,PASOS),activation='tanh'))
    model.add(Flatten())
    model.add(Dense(1, activation='tanh'))
    model.compile(loss='mean_absolute_error',optimizer='Adam',metrics=["mse"])
    model.summary()
    return model

In [ ]:
EPOCHS=40
 
model = crear_modeloFF()
 
history=model.fit(x_train,y_train,epochs=EPOCHS,validation_data=(x_val,y_val),batch_size=PASOS)

In [ ]:
X_test= X_test[7:] #Delete first 7 days of data test.

In [ ]:
results=model.predict(x_val)
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
plt.scatter(X_train['ds'], y_train, c= 'b')
plt.scatter(X_test['ds'],y_val,c='g')
plt.scatter(X_test['ds'],results,c='r')
plt.title('validate')
plt.show()

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
# summarize history for accuracy
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('model mse')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()